In [2]:
import os, shutil, yaml

# 이미지 파일 라벨링 파일에 맞게 train, test 파일 split하기

## 파일 리스트 만들기

In [3]:
img_path = "/home/erbl/data/MedicalAids/Images_RGB"
train_label_path = "/home/erbl/data/MedicalAids/Annotations_RGB"
test_label_path = "/home/erbl/data/MedicalAids/Annotations_RGB_TestSet2"

img_list = list(map(lambda x: os.path.splitext(x)[0], os.listdir(img_path)))
train_label_list = list(map(lambda x: os.path.splitext(x)[0], os.listdir(train_label_path)))
test_label_list = list(map(lambda x: os.path.splitext(x)[0], os.listdir(test_label_path)))

In [4]:
# train, test 비율
total, train, test = len(img_list), len(train_label_list), len(test_label_list)

print("test:", round(test/total*100), "%")
print("train: ", round(train/total*100), "%")

test: 11 %
train:  89 %


## 파일 생성 및 이동

In [45]:
img_base_path = "/home/erbl/data/MedicalAids/images"
os.makedirs(f"{img_base_path}/train", exist_ok=True)
os.makedirs(f"{img_base_path}/test", exist_ok=True)

In [47]:
destination = f"{img_base_path}/train"

for filename in train_label_list:
    new_path = shutil.copy(f"{img_path}/{filename}.png", destination)

In [46]:
destination = f"{img_base_path}/test"

for filename in test_label_list:
    new_path = shutil.copy(f"{img_path}/{filename}.png", destination)

# annotation 파일 형식 변환 (yml to txt)

In [60]:

# TEST
print(f"{train_label_path}/{train_label_list[0]}.yml")
with open(f"{train_label_path}/{train_label_list[0]}.yml") as f:
    annotations = yaml.load(f, Loader=yaml.FullLoader)
    filename = annotations['annotation']['filename']

print(annotations)
print(filename)

/home/erbl/data/MedicalAids/Annotations_RGB/seq_1477472233.1736589970.yml
{'annotation': {'filename': 'seq_1477472233.1736589970.png', 'folder': 'mobility-aids', 'object': [{'bndbox': {'depth': 2.96077036857605, 'xmax': '799', 'xmin': '669', 'ymax': '444', 'ymin': '80'}, 'name': 'person'}, {'bndbox': {'depth': 2.178972244262695, 'xmax': '445', 'xmin': '274', 'ymax': '540', 'ymin': '86'}, 'name': 'person'}], 'owner': 'Andres Vasquez', 'segmented': '0', 'size': {'depth': '3', 'height': '540', 'width': '960'}, 'source': '{database: Freiburg Mobility Aids Dataset}'}}
seq_1477472233.1736589970.png


In [14]:
# txt 형식의 annotation 파일을 담을 새로운 디렉토리 만들기
img_base_path = "/home/erbl/data/MedicalAids/labels"
os.makedirs(f"{img_base_path}/train", exist_ok=True)
os.makedirs(f"{img_base_path}/test", exist_ok=True)

## 새로운 형식의 파일 만들기

In [ ]:
# 클래스 인데싱 딕셔너리 -> 변경 필요
class_dir = {'person': 0, 'wheelchair': 1, 'push_wheelchair': 2, 'crutches': 3, 'walking_frame': 4} # 수정, 추가 필요

In [29]:
# 한 이미지의 annotation 정보들 바꾸어 리스트로 반환
def transform_annotation(annotations):

		results = []
		if 'object' in annotations['annotation']:

			objs = annotations['annotation']['object']
			height = int(annotations['annotation']['size']['height'])
			width = int(annotations['annotation']['size']['width'])

			for i in range(len(objs)): # i: bndbox 물체들의 인덱스
				xmax = int(objs[i]['bndbox']['xmax'])
				xmin = int(objs[i]['bndbox']['xmin'])
				ymax = int(objs[i]['bndbox']['ymax'])
				ymin = int(objs[i]['bndbox']['ymin'])
				
				# 중심값, normalized된 값
				center_x = (xmax + xmin) / (2 * width)
				center_y = (ymax + ymin) / (2 * height)
				bbox_width = (xmax - xmin) / width
				bbox_height = (ymax - ymin) / height

				# class 이름을 index로
				class_idx = class_dir[objs[i]['name']]
				results.append([class_idx, center_x, center_y, bbox_width, bbox_height])
			
		return results

In [13]:
# TEST
with open("/home/erbl/seq_1468843742.4620746090.yml") as f:
    annotations = yaml.load(f, Loader=yaml.FullLoader)
    new_annotations = transform_annotation(annotations)

print(new_annotations)

# 새로운 텍스트 파일 경로 설정 -> 변경 필요!!!!!
new_label_base_path = "/home/erbl/data/MedicalAids/labels/test" 

# txt 형태의 annotation 파일 만들기
with open(f"{new_label_base_path}/seq_1468843742.4620746090.txt", "w") as f:
    for annotation in new_annotations:
        line = " ".join(map(str, annotation)) + "\n"
        f.write(line)

[[0, 0.3151041666666667, 0.4583333333333333, 0.053125, 0.1537037037037037], [2, 0.26666666666666666, 0.4740740740740741, 0.04791666666666667, 0.35185185185185186], [2, 0.4244791666666667, 0.45925925925925926, 0.06770833333333333, 0.3592592592592593]]


In [21]:
# 새로운 텍스트 파일 경로 설정
new_label_base_path = "/home/erbl/data/MedicalAids/labels" 
print(f"'{test_label_path}'를 읽어 '{new_label_base_path}/test'에 새로운 포맷으로 저장")

'/home/erbl/data/MedicalAids/Annotations_RGB_TestSet2'를 읽어 '/home/erbl/data/MedicalAids/labels/test'에 새로운 포맷으로 저장


In [24]:
# test annotations
for label_path in test_label_list:
	with open(f"{test_label_path}/{label_path}.yml") as f:
		# yaml 파일에서 annotation 정보 로드받기
		annotations = yaml.load(f, Loader=yaml.FullLoader)
		filename = annotations['annotation']['filename']		

		# 새로운 annotation 값 계산
		new_annotations = transform_annotation(annotations)

	# txt 형태의 annotation 파일 만들기
	with open(f"{new_label_base_path}/test/{label_path}.txt", "w") as f:
		for annotation in new_annotations:
			line = " ".join(map(str, annotation)) + "\n"
			f.write(line)

In [27]:
# 새로운 텍스트 파일 경로 설정
print(f"'{train_label_path}'를 읽어 '{new_label_base_path}/train'에 새로운 포맷으로 저장")

'/home/erbl/data/MedicalAids/Annotations_RGB'를 읽어 '/home/erbl/data/MedicalAids/labels/train'에 새로운 포맷으로 저장


In [30]:
# train annotations
for label_path in train_label_list:
	with open(f"{train_label_path}/{label_path}.yml") as f:
		# yaml 파일에서 annotation 정보 로드받기
		annotations = yaml.load(f, Loader=yaml.FullLoader)
		# filename = annotations['annotation']['filename']		

		# 새로운 annotation 값 계산
		new_annotations = transform_annotation(annotations)

	# txt 형태의 annotation 파일 만들기
	with open(f"{new_label_base_path}/train/{label_path}.txt", "w") as f:
		for annotation in new_annotations:
			line = " ".join(map(str, annotation)) + "\n"
			f.write(line)